In [1]:
HIPCT_HEMI = "https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/"
HIPCT_IC2 = "https://neuroglancer.lincbrain.org/zarr/6d25507c-ae9d-4f61-9a92-8644783adfd9/"
HIPCT_HEMI2RAS = "https://neuroglancer.lincbrain.org/blobs/bb9/764/bb9764d6-e736-4332-bcfa-0c772e294ab4"
HIPCT_HEMI2IC2 = "https://neuroglancer.lincbrain.org/blobs/d5c/eb7/d5ceb784-193f-4fb8-b0a9-e5db57658ade"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ngtools.scene import Scene
import json

In [4]:
colormap_orient = """
vec3 colormapOrient(vec3 orient) {
  vec3 result;
  result.r = abs(orient[0]);
  result.g = abs(orient[1]);
  result.b = abs(orient[2]);
  return clamp(result, 0.0, 1.0);
}
"""

# Runing this in the skeleton shader uses orientation to color tracts
orient_shader = colormap_orient + """
#uicontrol bool orient_color checkbox(default=true)
void main() {
  vec3 orient;
  orient.r = getDataValue(0);
  orient.g = getDataValue(1);
  orient.b = getDataValue(2);
  if (orient_color)
    emitRGB(colormapOrient(orient));
  else
    emitDefault();
}
"""

def rotate_orient_shader(mat):
    return (colormap_orient +
"""
#uicontrol bool orient_color checkbox(default=true)
void main() {
"""
+
f"""
    mat3 mat = mat3(
      {mat[0, 0]}, {mat[1, 0]}, {mat[2, 0]},
      {mat[0, 1]}, {mat[1, 1]}, {mat[2, 1]},
      {mat[0, 2]}, {mat[1, 2]}, {mat[2, 2]}
    );
    vec3 orient;
    orient.r = getDataValue(0);
    orient.g = getDataValue(1);
    orient.b = getDataValue(2);
    vec3 orient_rot = mat * orient;
  if (orient_color)
    emitRGB(colormapOrient(orient_rot));
  else
    emitDefault();
"""
+
"""
}
"""
)

ic_shader = """
#uicontrol invlerp normalized
void main() {
  vec4 rgba;
  rgba.r = normalized();
  rgba.a = normalized();
  emitRGBA(rgba);
}
"""

In [5]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

scene = Scene()
scene.load({"HiP-CT (hemi)": "zarr://" + HIPCT_HEMI})
scene.load({"HiP-CT (internal capsule)": "zarr://" + HIPCT_IC2})
scene.transform("lta://" + HIPCT_HEMI2IC2, layer="HiP-CT (internal capsule)")
scene.transform("lta://" + HIPCT_HEMI2RAS, inv=True)


DEBUG:ngtools.scene:load({'HiP-CT (hemi)': 'zarr://https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/'})
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lincbrain.org:443
DEBUG:urllib3.connectionpool:https://api.lincbrain.org:443 "GET /api/auth/token HTTP/11" 301 None
DEBUG:urllib3.connectionpool:https://api.lincbrain.org:443 "GET /api/auth/token/ HTTP/11" 200 42
DEBUG:urllib3.connectionpool:https://api.lincbrain.org:443 "GET /api/permissions/s3/ HTTP/11" 200 44
DEBUG:asyncio:Using selector: KqueueSelector
DEBUG:fsspec.http:https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/zarr.json
DEBUG:ngtools.opener:exists(https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/zarr.json): 1.458939790725708 s
DEBUG:fsspec.http:https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/.zarray
DEBUG:ngtools.opener:exists(https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f

In [6]:
print(json.dumps(scene.to_json(), indent=4))

{
    "layers": [
        {
            "type": "image",
            "source": [
                {
                    "url": "zarr2://https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61",
                    "transform": {
                        "matrix": [
                            [
                                1.0,
                                0.0,
                                0.0,
                                -7.565
                            ],
                            [
                                0.0,
                                1.0,
                                0.0,
                                -7.565
                            ],
                            [
                                0.0,
                                0.0,
                                1.0,
                                -7.565
                            ]
                        ],
                        "inputDimensions": {
        

In [7]:
scene.transform("lta://" + HIPCT_HEMI2IC2, layer="HiP-CT (internal capsule)")
scene.transform("lta://" + HIPCT_HEMI2RAS, inv=True)

DEBUG:ngtools.scene:transform("lta://https://neuroglancer.lincbrain.org/blobs/d5c/eb7/d5ceb784-193f-4fb8-b0a9-e5db57658ade", layer="HiP-CT (internal capsule)")
DEBUG:ngtools.scene:world_axes()
DEBUG:ngtools.scene:world_axes({'x': 'x', 'y': 'y', 'z': 'z'})
DEBUG:ngtools.scene:rename_axes({'x': 'x', 'y': 'y', 'z': 'z', 't': 't'})
DEBUG:ngtools.scene:rename_axes({'x': 'x', 'y': 'y', 'z': 'z', 't': 't'})
DEBUG:fsspec.http:Retrieve file size for https://neuroglancer.lincbrain.org/blobs/d5c/eb7/d5ceb784-193f-4fb8-b0a9-e5db57658ade
DEBUG:fsspec.http:Fetch range for <File-like object HTTPFileSystem, https://neuroglancer.lincbrain.org/blobs/d5c/eb7/d5ceb784-193f-4fb8-b0a9-e5db57658ade>: 0-201
DEBUG:fsspec.http:https://neuroglancer.lincbrain.org/blobs/d5c/eb7/d5ceb784-193f-4fb8-b0a9-e5db57658ade : bytes=0-200
DEBUG:fsspec:<File-like object HTTPFileSystem, https://neuroglancer.lincbrain.org/blobs/d5c/eb7/d5ceb784-193f-4fb8-b0a9-e5db57658ade> read: 0 - 201  , bytes: 0 hits, 1 misses, 201 total req

CoordinateSpaceTransform({"matrix": [[0.6996975, 0.6888085, -0.15023576, 43.315273], [0.11950177, -0.32448298, -0.93114716, 100.78205], [-0.69480044, 0.6378544, -0.31144667, 81.25659]], "inputDimensions": {"x": [1.0, "mm"], "y": [1.0, "mm"], "z": [1.0, "mm"]}, "outputDimensions": {"x": [1.0, "mm"], "y": [1.0, "mm"], "z": [1.0, "mm"]}})
layers before [ManagedLayer("HiP-CT (hemi)",{"type": "image", "source": [{"url": "zarr2://https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61", "transform": {"matrix": [[1.0, 0.0, 0.0, -7.565], [0.0, 1.0, 0.0, -7.565], [0.0, 0.0, 1.0, -7.565]], "inputDimensions": {"z": [15.13, "um"], "y": [15.13, "um"], "x": [15.13, "um"]}, "outputDimensions": {"z": [15.13, "um"], "y": [15.13, "um"], "x": [15.13, "um"]}}}], "shaderControls": {"normalized": {"range": [23005.0, 27807.0], "window": [3412.0, 57812.0]}}, "name": "HiP-CT (hemi)"}), ManagedLayer("__world_axes_native__",{"type": "annotation", "source": [{"url": "local://annotations", 

DEBUG:fsspec.http:Fetch range for <File-like object HTTPFileSystem, https://neuroglancer.lincbrain.org/blobs/bb9/764/bb9764d6-e736-4332-bcfa-0c772e294ab4>: 0-1501
DEBUG:fsspec.http:https://neuroglancer.lincbrain.org/blobs/bb9/764/bb9764d6-e736-4332-bcfa-0c772e294ab4 : bytes=0-1500
DEBUG:fsspec:<File-like object HTTPFileSystem, https://neuroglancer.lincbrain.org/blobs/bb9/764/bb9764d6-e736-4332-bcfa-0c772e294ab4> read: 0 - 1501  , bytes: 0 hits, 1 misses, 1501 total requested bytes
DEBUG:ngtools.transforms:Succesfully read format "lta".
DEBUG:ngtools.scene:world_axes({'x': 'x', 'y': 'y', 'z': 'z', 't': 't'})
DEBUG:ngtools.scene:rename_axes({'x': 'x', 'y': 'y', 'z': 'z', 't': 't'})
DEBUG:ngtools.scene:rename_axes({'x': 'x', 'y': 'y', 'z': 'z', 't': 't'})
DEBUG:ngtools.scene:display(None)


CoordinateSpaceTransform({"matrix": [[-0.9698841822671607, 0.09338922770402579, -0.22495106123789232, 151.65965605479983], [0.24356226792145788, 0.3771409220197704, -0.893555884953376, 98.918861831316], [0.0013897606983883917, -0.9214353440515287, -0.3885291710817049, 177.24598576864454]], "inputDimensions": {"x": [1.0, "mm"], "y": [1.0, "mm"], "z": [1.0, "mm"]}, "outputDimensions": {"x": [1.0, "mm"], "y": [1.0, "mm"], "z": [1.0, "mm"]}})
layers before [ManagedLayer("HiP-CT (hemi)",{"type": "image", "source": [{"url": "zarr2://https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61", "transform": {"matrix": [[1.0, 0.0, 0.0, -7.565], [0.0, 1.0, 0.0, -7.565], [0.0, 0.0, 1.0, -7.565]], "inputDimensions": {"z": [15.13, "um"], "y": [15.13, "um"], "x": [15.13, "um"]}, "outputDimensions": {"z": [15.13, "um"], "y": [15.13, "um"], "x": [15.13, "um"]}}}], "shaderControls": {"normalized": {"range": [23005.0, 27807.0], "window": [3412.0, 57812.0]}}, "name": "HiP-CT (hemi)"}

In [8]:
print(json.dumps(scene.to_json(), indent=4))

{
    "layers": [
        {
            "type": "image",
            "source": [
                {
                    "url": "zarr2://https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61",
                    "transform": {
                        "inputDimensions": {
                            "z": [
                                15.13,
                                "um"
                            ],
                            "y": [
                                15.13,
                                "um"
                            ],
                            "x": [
                                15.13,
                                "um"
                            ]
                        },
                        "outputDimensions": {
                            "x": [
                                1.0,
                                "mm"
                            ],
                            "y": [
                              